In [ ]:
rm(list = ls())
setwd("/home/creambbq/facu/Datos de panel/TP4")
library("haven"); library("plm"); library("dplyr"); library("pglm");
set.seed(1313)

In [2]:
get_data <- function(N, TT, model) {
  df <- df <- setNames(data.frame(matrix(0, ncol = 10, nrow = N*TT)), 
                       c("j", "t", "x", "z", "eps", "u", 
                         "psi_1", "psi_2", "psi_3", "psi_4"))
  aux <- 1
  for(j in 1:N){
    psi_2 <- rnorm(1,0,1)
    psi_3 <- rnorm(1,0,1)
    psi_4 <- rnorm(1,0,1)
    for(t in 1:TT){
      x <- rnorm(1,0,1)
      z <- rnorm(1,0,1)
      eps <- rnorm(1,0,1)
      psi_1 <- rnorm(1,0,1)
      u <- 0.6*eps + 0.8*psi_1
      df[aux, ] <- c(j, t, x, z, eps, u, psi_1, psi_2, psi_3, psi_4)
      aux <- aux + 1
    }
  }
  if (model == "A"){
    df <- df %>% mutate(alpha = psi_2 + psi_4, 
                        c = psi_3 + psi_4, 
                        s = case_when(x + z + alpha + eps > 0 ~ 1,
                                      TRUE ~ 0),
                        y = case_when(s == 1 ~ x + c + u,
                                      TRUE ~ NA_real_))
  } else if (model == "B"){
    df <- df %>% group_by(j) %>% mutate(alpha = psi_2 + sum(z)/2, 
                                        c = psi_3 + sum(x)/2) %>% 
      ungroup()
    df <- df %>% mutate(s = case_when(x + z + alpha + eps > 0 ~ 1,
                                      TRUE ~ 0),
                        y = case_when(s == 1 ~ x + c + u,
                                      TRUE ~ NA_real_))
  } else if (model == "C") { 
    df <- df %>% group_by(j) %>% mutate(alpha = psi_2 + sum(z)/2 + psi_4, 
                                        c = psi_3 + sum(x)/2 + psi_4) %>% 
      ungroup()
    df <- df %>% mutate(s = case_when(x + z + alpha + eps > 0 ~ 1,
                                      TRUE ~ 0),
                        y = case_when(s == 1 ~ x + c + u,
                                      TRUE ~ NA_real_))
    }
  df <- df[, c("j", "t", "y", "x", "s", "z", "alpha", "c")]
  return(df)
}
get_wooldridge <- function(df, boots){
  df <- df %>% group_by(j) %>% mutate(mean_x = mean(x), 
                                      mean_z = mean(z), 
                                      t1 = case_when(t == 1 ~ 1,
                                                     TRUE ~ 0), 
                                      t2 = case_when(t == 2 ~ 1, 
                                                     TRUE ~ 0)) %>% 
    ungroup()
  pdata <- pdata.frame(df, index = c("j", "t"))
  probit <- pglm(s ~ x + mean_x + z + mean_z -1, 
                 family = binomial("probit"), 
                 model = "pooling", 
                 method = "bfgs",
                 data = pdata)
  df <- df %>% mutate(pred = probit$estimate[1]*x + 
                        probit$estimate[2]*mean_x + 
                        probit$estimate[3]*z + 
                        probit$estimate[4]*mean_z, 
                      lambda = dnorm(pred)/pnorm(pred), 
                      lambda_1 = t1*lambda,
                      lambda_2 = t2*lambda)
  pdata <- pdata.frame(df, index = c("j", "t"))
  pOls <- plm(y ~ x + mean_x + lambda_1 + lambda_2 -1,
              fixed = c("j", "t"), 
              effect = "individual", 
              model = "pooling", 
              data = pdata)
  if (boots){
    return(c(pOls$coefficients[1], probit$estimate[1], probit$estimate[3],(df$s-df$pred)))
  } else {
    return(c(pOls$coefficients[1], probit$estimate[1], probit$estimate[3]))
  }
}
montecarlo <- function(S) {
  models <- c("A", "B", "C")
  Ns <- c(20,40,100)
  df <- df <- setNames(data.frame(matrix(0, ncol = 6, nrow = length(Ns)*S)),
                       c("N", "T", "model", "beta", "gamma_1", "gamma_2"))
  aux <- 1
  for(s in 1:S) {
    for(N in Ns){
      for(model in models){
        data <- get_data(N, 2, model)
        df[aux, ] <- c(N, 2, model, get_wooldridge(data, boots = FALSE))
        aux <- aux + 1
      }
    }
  }
  resultados <- df %>% mutate(across(!model, as.numeric)) %>% 
    group_by(N, model) %>% 
    summarise(sesgo_medio_beta = mean(beta) - 1, 
              sesgo_medio_gamma1 = mean(gamma_1) - 1, 
              sesgo_medio_gamma2 = mean(gamma_2) - 1, 
              sesgo_mediano_beta = median(beta) - 1,
              sesgo_mediano_gamma1 = median(gamma_1) - 1,
              sesgo_mediano_gamma2 = median(gamma_2) - 1,
              desvio_beta = sqrt((sum(beta - mean(beta))^2)/S), 
              desvio_gamma1 = sqrt((sum(gamma_1 - mean(gamma_1))^2)/S),
              desvio_gamma2 = sqrt((sum(gamma_2 - mean(gamma_2))^2)/S), 
              rmse_beta = sqrt(((sum(mean(beta)-1))^2)/S), 
              rmse_gamma_1 = sqrt(((sum(mean(gamma_1)-1))^2)/S), 
              rmse_gamma_2= sqrt(((sum(mean(gamma_2)-1))^2)/S), 
              desvio_medio_abs_beta = abs(sum(beta-mean(beta)))/S, 
              desvio_medio_abs_gamma_1= abs(sum(beta-mean(beta)))/S,
              desvio_medio_abs_gamma_2 = abs(sum(beta-mean(beta)))/S)
  return(resultados)
}
get_bt_iteration <- function(N, TT, model){
  df <- get_data(N, TT, model)
  wold <- get_wooldridge(df, boots = TRUE)
  beta_hat <- wold[1]
  gamma1_hat <- wold[2]
  gamma2_hat <- wold[3]
  eps_hat <- wold[4:(N*TT+3)]
  muestra <- eps_hat[sample.int(N*TT, N*TT, replace = TRUE)]
  df["eps_hat"] <- muestra
  df <- df %>% mutate(s = case_when(gamma1_hat*x + gamma2_hat*z + alpha + eps_hat > 0 ~ 1,
                                    TRUE ~ 0),
                      y = case_when(s == 1 ~ beta_hat*x + c,
                                    TRUE ~ NA_real_))
  return(get_wooldridge(df, boots = FALSE))
}
bootstrap <- function(B){
  models <- c("A", "B", "C")
  Ns <- c(20,40,100)
  res <- setNames(data.frame(matrix(0, ncol = 6, nrow = B*length(Ns))), 
                       c("N", "T", "model", "beta", "gamma1", "gamma2"))
  aux <- 1
  for (b in 1:B){
    for (N in Ns){
      for(model in models){
        res[aux, ] <- c(N, 10, model, get_bt_iteration(N, 10, model))
        aux <- aux + 1
      }
    }
  }
  res <- res %>% mutate(across(!model, as.numeric)) %>% 
    group_by(N, model) %>% 
    summarise(beta = paste(t.test(beta, conf.level = 0.95)$conf.int[1], 
                           t.test(beta, conf.level = 0.95)$conf.int[2], 
                                                    sep = " - "), 
              gamma1 = paste(t.test(gamma1, conf.level = 0.95)$conf.int[1], 
                             t.test(gamma1, conf.level = 0.95)$conf.int[2], 
                             sep = " - "), 
              gamma2 = paste(t.test(gamma2, conf.level = 0.95)$conf.int[1], 
                             t.test(gamma2, conf.level = 0.95)$conf.int[2], 
                             sep = " - "))
  return(res)
}

In [3]:
montecarlo(S = 1000)

`summarise()` has grouped output by 'N'. You can override using the `.groups`
argument.


N,model,sesgo_medio_beta,sesgo_medio_gamma1,sesgo_medio_gamma2,sesgo_mediano_beta,sesgo_mediano_gamma1,sesgo_mediano_gamma2,desvio_beta,desvio_gamma1,desvio_gamma2,rmse_beta,rmse_gamma_1,rmse_gamma_2,desvio_medio_abs_beta,desvio_medio_abs_gamma_1,desvio_medio_abs_gamma_2
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
20,A,-0.013584633,-0.24315471,-0.27373559,0.001893255,-0.3641324,-0.3654633,1.184906e-15,1.155064e-15,5.213588e-16,4.295838e-04,0.007689227,0.008656280,3.747003e-17,3.747003e-17,3.747003e-17
20,B,0.016531113,0.03236367,0.09009396,0.006910330,-0.1655959,-0.1723873,2.468116e-15,2.748983e-15,3.173793e-15,5.227597e-04,0.001023429,0.002849021,7.804868e-17,7.804868e-17,7.804868e-17
20,C,-0.005776740,-0.21451050,-0.25371726,0.005905565,-0.3462964,-0.3562652,6.705692e-16,2.106500e-16,5.617334e-16,1.826766e-04,0.006783418,0.008023244,2.120526e-17,2.120526e-17,2.120526e-17
40,A,-0.002632930,-0.36887543,-0.36997998,0.006407633,-0.3876754,-0.3923405,4.581638e-16,8.022254e-16,1.065538e-15,8.326057e-05,0.011664865,0.011699794,1.448841e-17,1.448841e-17,1.448841e-17
40,B,0.000558407,-0.21327129,-0.22583979,-0.009409462,-0.2493248,-0.2552890,3.826808e-16,3.510833e-18,1.713287e-15,1.765838e-05,0.006744230,0.007141681,1.210143e-17,1.210143e-17,1.210143e-17
40,C,0.012841979,-0.36618156,-0.37417363,0.014853030,-0.3910069,-0.3981727,3.170283e-15,9.022842e-16,1.095380e-15,4.060990e-04,0.011579678,0.011832409,1.002531e-16,1.002531e-16,1.002531e-16
100,A,0.007157953,-0.41022104,-0.40489360,0.007583514,-0.4134851,-0.4107141,2.973676e-15,1.753661e-15,2.773558e-16,2.263544e-04,0.012972328,0.012803860,9.403589e-17,9.403589e-17,9.403589e-17
100,B,-0.002137657,-0.26151629,-0.26674519,-0.011041827,-0.2674509,-0.2783709,1.091869e-15,1.372736e-15,1.583386e-15,6.759864e-05,0.008269871,0.008435223,3.452794e-17,3.452794e-17,3.452794e-17
100,C,0.003199397,-0.40208498,-0.40378794,0.000075291,-0.4105895,-0.4121877,2.159163e-15,1.618494e-15,1.586897e-15,1.011738e-04,0.012715044,0.012768896,6.827872e-17,6.827872e-17,6.827872e-17


In [15]:
bootstrap(B = 1000)

`summarise()` has grouped output by 'N'. You can override using the `.groups`
argument.


N,model,beta,gamma1,gamma2
<dbl>,<chr>,<chr>,<chr>,<chr>
20,A,0.942726726662758 - 0.974482112022787,0.386125019480433 - 0.403031088778777,0.382355251677562 - 0.399844054950864
20,B,0.983825601804865 - 1.00696275370419,0.4213169985221 - 0.439095522841933,0.414390225560934 - 0.432836033179865
20,C,0.97450692678223 - 1.0020143508821,0.326951818960078 - 0.344049703093458,0.329376065797398 - 0.34657992870442
40,A,0.962441259867782 - 0.983173414351192,0.359411627448733 - 0.371437740605436,0.359955157604336 - 0.37208840547083
40,B,0.987416317209452 - 1.00345752398281,0.411808109584478 - 0.424621621317865,0.413198934519474 - 0.4252195181738
40,C,0.975269889419827 - 0.99496729353996,0.322627867851751 - 0.334721296332131,0.321170714880369 - 0.332914296847212
100,A,0.963954964507596 - 0.977399854051773,0.354512251462243 - 0.361790500751028,0.353532539710579 - 0.360670011326123
100,B,0.988259456439461 - 0.998416239351141,0.40271729179876 - 0.410557879800307,0.404124409095184 - 0.412233967561336
100,C,0.986255820549675 - 0.998463534022802,0.313618671780353 - 0.321025346000642,0.316842028882268 - 0.324286437458612
